In [ ]:
import getpass
import time
import os
import json
import pickle
import pandas as pd
import numpy as np
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [2]:
if not os.environ.get("TOGETHER_API_KEY"):
    os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter API key for Together AI: ")

llm = init_chat_model("meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", model_provider="together")

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Users/hoangvinh/.pyenv/versions/3.10.6/envs/venv_motel_room/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
all_data = pd.read_csv('../EDA/final_data.csv')
all_data.head(3)

,merge_file_id,source_post,link,title,price,area,describe,address,province_city,district,ward,street,house_number
0,p1-l0_20250320-000509,batdongsan,https://batdongsan.com.vn/cho-thue-nha-tro-pho...,Khai trương phòng Nguyễn Duy Trinh giá 3tr8.Ch...,3.8,22.0,"\n Phòng cho thuê tại Quận 2, H...","Đường Nguyễn Duy Trinh, Phường Bình Trưng Đông...",Hồ Chí Minh,Thủ Đức,Bình Trưng Đông,Đường Nguyễn Duy Trinh,NaN
1,p1-l10_20250320-000536,batdongsan,https://batdongsan.com.vn/cho-thue-nha-tro-pho...,\n Chính chủ cho thuê phòng...,5.9,35.0,\n Phòng trọ nằm ở 15B/105 ...,"15B/105 Đường Lê Thánh Tôn, Phường Bến Nghé, Q...",Hồ Chí Minh,1,Bến Nghé,15B/105 Đường Lê Thánh Tôn,NaN
2,p1-l11_20250320-000536,batdongsan,https://batdongsan.com.vn/cho-thue-nha-tro-pho...,\n Khai trương CHDV full nộ...,7.0,40.0,"\n Tô Hiến Thành, P13, Q10....","Đường Tô Hiến Thành, Phường 13, Quận 10, Hồ Ch...",Hồ Chí Minh,10,13,Đường Tô Hiến Thành,NaN


In [5]:
data_dict = all_data.to_dict(orient='records')

In [ ]:
data_text_dict = {}
for data in data_dict:
    data_text_dict[data["merge_file_id"]] = json.dumps(
        obj={
            "id": data["merge_file_id"],
            "address": data["address"],
            "area": data["area"],
            "price": data["price"],
            "describe": data["describe"]
        }, 
        ensure_ascii=False
    )

documents = []
for id, data_text in data_text_dict.items():
    documents.append(
        Document(
            page_content=data_text,
            metadata={'id': id}
        )
    )

In [37]:
data_text_df = pd.DataFrame(list(data_text_dict.items()), columns=['id', 'text'])
data_text_df.to_csv('data_text.csv', index=False)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "!", "?", ",", "\n", "\n\n"],
    chunk_size=100,
    chunk_overlap=50
)
all_splits = text_splitter.split_documents(documents)

In [9]:
# Tạo FAISS index
vector_store = FAISS.from_documents(all_splits, embeddings)

In [10]:
with open('vector_store.pkl', 'wb') as file:
    pickle.dump(vector_store, file)

In [11]:
query = 'Căn trọ cho nuôi chó mèo'

In [12]:
results = vector_store.similarity_search(query, k=20)

In [ ]:
result_ids = [doc.metadata['id'] for doc in results]

['p74-l1_20250321-114439',
 'p197-l12_20250404-173602',
 'p33-l2_20250320-175230',
 'p5-l7_20250320-011409',
 'p73-l11_20250321-114439',
 'p6-l1_20250320-172724',
 'p1-l4_20250320-000536',
 'p28-l14_20250320-175230',
 'p34-l1_20250320-175230',
 'p88-l5_20250321-120549',
 'p69-l10_20250404-160805',
 'p135-l0_20250404-172843',
 'p32-l14_20250320-175230',
 'p40-l11_20250321-080639',
 'p2-l2_20250320-011409',
 'p50-l14_20250321-091109',
 'p37-l14_20250320-175230',
 'p37-l7_20250320-175230',
 'p31-l12_20250320-175230',
 'p25-l4_20250320-175230']

In [24]:
system_prompt = """Bạn là một trợ lý tìm kiếm bất động sản.
Sử dụng thông tin dưới đây (dạng JSON) để trả lời câu hỏi ngắn gọn dưới 3 câu.
Nếu có thể, tạo một list ID các mã như ["id1", "id2", ...].
Nếu không tìm thấy thông tin liên quan, hãy trả lời [].

Thông tin:\n{retrieved_text}"""

custom_rag_prompt = PromptTemplate.from_template(system_prompt)

In [ ]:
result_data_text = data_text.loc[data_text['id'].isin(result_ids), 'text'].to_list()
retrieved_text = ",\n".join(result_data_text)
formatted_prompt = custom_rag_prompt.format(retrieved_text=retrieved_text)

In [26]:
final_answer = llm.invoke([
    SystemMessage(content=formatted_prompt),
    HumanMessage(content=query)
])

In [29]:
filter_ids = json.loads(final_answer.content)

In [30]:
all_data.loc[all_data['merge_file_id'].isin(filter_ids), 'link'].to_list()

['https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-tran-van-hoang-phuong-9-13/cho-gia-re-gan-cu-xa-bac-hai-ly-thuong-kiet-q10-full-noi-that-mien-phi-2-xe-pr42428058',
 'https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-duong-so-38-phuong-hiep-binh-chanh/cho-full-noi-that-ngay-pham-van-dong-gan-gigamall-linh-dong-cau-loi-pr42068180',
 'https://thuviennhadat.vn/nha-dat-thue-chi-tiet/can-ho-full-noi-that-ban-cong-mat-truoc-gan-dh-cong-thuong-go-truong-chinh-18711.html',
 'https://thuviennhadat.vn/nha-dat-thue-chi-tiet/cho-thue-phong-sinh-vien-moi-xay-ngay-co-so-iuh-moi-nguyen-van-dung-17407.html',
 'https://thuviennhadat.vn/nha-dat-thue-chi-tiet/cho-thue-phong-ban-cong-35m2-duong-hoang-sa-phuong-vo-thi-sau-quan-3-gia-5-900-000-vnd-21019.html',
 'https://www.nhatot.com/thue-phong-tro-quan-4-tp-ho-chi-minh/123630625.htm']